## Goal
* Gather sunset and sunrise times from an array of dates and locations and store results

https://sunrise-sunset.org/api

In [2]:
# Import tools
from urllib.request import urlopen
from datetime import timedelta, date, datetime
from dateutil import tz
import json
import pandas as pd
import time
import os.path

## Build Date Array

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Initialize date range start at 6/30 to compensate for UTC
start_date = date(2018, 6, 30)
end_date = date(2017, 11, 1)

dates = []

for single_date in daterange(start_date, end_date):
        dates.append(single_date.strftime("%Y-%m-%d"))

In [3]:
print(dates[:4])
print(dates[-4:])

[]
[]


## Load Location Dictionary

In [3]:
# Load station location csv into a DataFrame
stations = pd.read_csv('../output/mesoweb-station_meta-180919.csv')

In [5]:
stations.head()

,Unnamed: 0,COUNTRY,COUNTY,CWA,ELEVATION,ELEV_DEM,GACC,ID,LATITUDE,LONGITUDE,...,NWSZONE,PERIOD_OF_RECORD,RESTRICTED,SGID,SHORTNAME,STATE,STATUS,STID,TIMEZONE,WIMS_ID
0,0,US,Ravalli,MSO,4420,4432.4,NRCC,2917,45.815556,-114.258333,...,MT006,"{'start': '2001-05-10T00:00:00Z', 'end': '2018...",False,NR06,RAWS,MT,ACTIVE,WSFM8,America/Denver,242907.0
1,1,US,Ravalli,MSO,4570,4767.1,NRCC,13522,45.820728,-113.953831,...,MT006,"{'start': '2005-03-25T00:00:00Z', 'end': '2018...",False,NR06,RAWS,MT,ACTIVE,SUAM8,America/Denver,242905.0
2,2,US,Ravalli,MSO,4500,4540.7,NRCC,3407,45.982667,-114.216250,...,MT005,"{'start': '2001-07-14T00:00:00Z', 'end': '2018...",False,NR06,RAWS,MT,ACTIVE,TR285,America/Denver,NaN
3,3,US,Ravalli,MSO,5507,5554.5,NRCC,6477,46.037828,-114.262708,...,MT005,"{'start': '2003-05-14T00:00:00Z', 'end': '2018...",False,NR06,RAWS,MT,ACTIVE,LRCM8,America/Denver,242914.0
4,4,US,Ravalli,MSO,7282,7257.2,NRCC,2856,46.025492,-114.055208,...,MT006,"{'start': '2001-06-01T00:00:00Z', 'end': '2018...",False,NR06,RAWS,MT,ACTIVE,DEEM8,America/Denver,242902.0


## Build URLs

In [ ]:
def build_url_df(stations, dates): 
    url_station = []
    # loop through stations in weather_monitoring-locations.csv
    for index, row in stations.iterrows():
        # loop through date range set above
        for date in dates:
            row_id = row['STID']
            lat = row['LATITUDE']
            lon = row['LONGITUDE']
            row_url = 'https://api.sunrise-sunset.org/json?lat=' + str(lat) + '&lng=' + str(lon) + '&date=' + date + '&formatted=0'
            url_station.append(dict({'station_id': row_id, 'url': row_url, 'date': date}))
    return(url_station)

In [ ]:
# Generate url list for api calls
url_batch = build_url_df(stations, dates)

In [ ]:
print('Number of urls: ' + str(len(url_batch)))
print('----\n', url_batch[-1:])

## Call API and store results

In [ ]:
def api_call(url):
    f = urlopen(url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    
    return parsed_json['results']

In [ ]:
def save_results(df_day, filepath):
    if os.path.isfile(filepath):
        with open(filepath, 'a') as f:
            df_day.to_csv(f, header=False, index=False)
    else:
        df_day.to_csv(filepath, index=False)

In [ ]:
%%time
successfull_calls = 0

# Loop over url_batch to call api and store results
for url in url_batch:
    # sleep to limit API call speed
    #time.sleep(3)
    
    print(url['station_id'] + '\n' + url['url'])
    result = api_call(url['url'])
    
    # include station_id
    result['station_id'] = url['station_id']
    
    # include date 
    result['date'] = url['date']
    
    df_day = pd.DataFrame.from_dict([result])

    save_results(df_day, '../output/sunrise_sunset-mesowest-utc.csv')
    
    successfull_calls += 1
    print('successfull_calls: ' + str(successfull_calls))

#### Additional Resources
[USNO](http://aa.usno.navy.mil/data/docs/RS_OneYear.php)